# Problem Statement:

Designing an end-to-end application that orchestrates the workflow between:

An orchestrator component that manages the overall process
A synthesizer component that generates specific outputs based on instructions

For our use case, let's create a Technical Documentation Generator that:

Takes high-level requirements as input
Breaks them down into specific documentation sections
Generates comprehensive technical documentation with code examples
Iteratively refines the documentation based on feedback

**Explanation of Approach**
My approach to this Langgraph application focuses on several key aspects:

**Orchestrator-Synthesizer Architecture**:

The orchestrator manages the workflow and delegates tasks
The synthesizer generates content based on specific instructions
I've added a reviewer component to provide feedback and ensure quality


**State Management:**

Using Langgraph's StateGraph to maintain document state throughout generation
Clear state transitions between planning, generating, reviewing, and refining


**Open Source Model Integration:**

Primary implementation using Ollama with local open-source models (Llama and Mistral)
Fallback to other models if local models aren't available


**Debugging with Langsmith:**

Comprehensive tracing of all operations
Detailed metrics for performance analysis
Tools for identifying bottlenecks and errors


**Iterative Refinement:**

The workflow includes feedback loops for content improvement
Maximum of 2 refinement iterations to prevent infinite loops


### **🚀 Steps for Implementing the Orchestrator and synthesizer**

 **Step 1: Set Up the Environment**
   - Install required libraries: `langchain`, `langgraph`, `transformers`, `huggingface_hub`, `langsmith`.
   - Configure LangSmith for debugging and monitoring.
  
  **!pip install -r requirements.txt**

In [1]:
import os
from typing import Dict, List, Tuple, TypedDict, Annotated, Literal
import json
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langsmith import Client
from langchain_core.tracers import LangChainTracer
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.tracers.langchain import wait_for_all_tracers
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Initialize tracer
tracer = LangChainTracer(project_name="technical-doc-project")
callback_manager = CallbackManager([tracer])


import os
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"  # Ensure the correct endpoint
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")  # Ensure the API key is set

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

langsmith_client=Client()

**Step2: Integrate Open-Source LLM**
   - Use **OLLAMA** (e.g., `llama3:8b`).


In [2]:
planner_model = ChatOpenAI(model="gpt-4o")
synthesizer_model = ChatOpenAI(model="gpt-4o")
reviewer_model = ChatOpenAI(model="gpt-4o")

In [3]:
# Define state schema
class DocumentState(TypedDict):
    # Input and requirements
    requirements: str
    # Planning
    sections: List[Dict[str, str]]
    section_index: int
    # Content generation
    current_section: Dict[str, str]
    section_content: Dict[str, str]
    # Feedback and iteration
    feedback: List[str]
    iteration_count: int
    # Final output
    complete_document: str
    # Flow control
    status: Literal["planning", "generating", "reviewing", "refining", "complete"]



In [4]:
# Define the nodes
# 1. Planner Node - Breaks down requirements into sections
from langsmith import trace

def plan_document(state: DocumentState) -> DocumentState:
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""You are a technical documentation planner. 
        Your task is to break down high-level requirements into specific document sections.
        Each section should have a title, purpose, and brief description of what it should contain."""),
        HumanMessage(content=f"Please create a detailed outline for technical documentation based on these requirements: {state['requirements']}")
    ])

    response = planner_model.invoke(prompt.format_messages())

    # 🔍 Debugging: Print LLM response
    print("\n🔎 Planner LLM Response:\n", response.content)

    # If the response is empty, return an error message
    if not response.content.strip():
        return {**state, "sections": [{"title": "Error", "description": "No response from LLM", "status": "error"}]}

    try:
        raw_sections = response.content.split("## ")[1:]  # Split into sections
        sections = []

        for section in raw_sections:
            lines = section.strip().split("\n")
            title = lines[0].strip()
            description = "\n".join(lines[1:]).strip()

            sections.append({
                "title": title,
                "description": description,
                "status": "pending"
            })

    except Exception as e:
        sections = [{"title": "Error in section parsing", 
                     "description": f"Error: {str(e)}\nRaw response: {response.content}",
                     "status": "error"}]

    return {**state, "sections": sections, "section_index": 0, "status": "generating"}



# 2. Synthesizer Node - Generates content for each section
def generate_section_content(state: DocumentState) -> DocumentState:
    if state["section_index"] >= len(state["sections"]):
        return {**state, "status": "complete"}

    current_section = state["sections"][state["section_index"]]

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""You are a technical documentation writer.
        Create comprehensive, clear content for the assigned section.
        Include code examples where appropriate.
        Format using markdown with proper headers, code blocks, and explanations."""),
        HumanMessage(content=f"""
        Project Requirements: {state['requirements']}
        
        Create content for the following section:
        
        ## {current_section['title']}
        
        Description: {current_section['description']}
        """)
    ])

    response = synthesizer_model.invoke(prompt.format_messages())

    # 🔍 Debugging: Print Synthesizer Response
    print(f"\n🔎 Synthesizer LLM Response for Section '{current_section['title']}':\n", response.content)

    if not response.content.strip():
        response.content = "⚠️ Error: No content generated."

    # Store the generated content
    section_content = state.get("section_content", {})
    section_content[current_section["title"]] = response.content

    return {
        **state,
        "current_section": current_section,
        "section_content": section_content,
        "status": "reviewing"
    }


# 3. Reviewer Node - Reviews the content and provides feedback
def review_section(state: DocumentState) -> DocumentState:
        current_section = state["current_section"]
        section_content = state["section_content"][current_section["title"]]
        
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a technical documentation reviewer.
            Evaluate the content provided for accuracy, clarity, and completeness.
            Provide specific, actionable feedback for improvements."""),
            HumanMessage(content=f"""
            Project Requirements: {state['requirements']}
            
            Section Purpose: {current_section['description']}
            
            Content to Review:
            
            {section_content}
            
            Provide feedback on this content:
            """)
        ])
        
        response = reviewer_model.invoke(prompt.format_messages())
        
        feedback = response.content
        current_feedback = state.get("feedback", [])
        current_feedback.append(feedback)
        
        # Check if feedback contains substantial change requests
        needs_refinement = "improve" in feedback.lower() or "revise" in feedback.lower() or "change" in feedback.lower()
        iteration_count = state.get("iteration_count", 0)
        
        if needs_refinement and iteration_count < 2:  # Limit to 2 refinement iterations
            return {
                **state,
                "feedback": current_feedback,
                "iteration_count": iteration_count + 1,
                "status": "refining"
            }
        else:
            # Move to the next section
            sections = state["sections"]
            sections[state["section_index"]]["status"] = "completed"
            
            return {
                **state,
                "feedback": current_feedback,
                "sections": sections,
                "section_index": state["section_index"] + 1,
                "iteration_count": 0,
                "status": "generating" if state["section_index"] + 1 < len(state["sections"]) else "complete"
            }

# 4. Refiner Node - Refines content based on feedback
def refine_section(state: DocumentState) -> DocumentState:
        current_section = state["current_section"]
        section_content = state["section_content"][current_section["title"]]
        latest_feedback = state["feedback"][-1]
        
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a technical documentation writer.
            Revise the content based on the feedback provided.
            Maintain markdown formatting with proper headers, code blocks, and explanations."""),
            HumanMessage(content=f"""
            Original Content:
            
            {section_content}
            
            Feedback Received:
            
            {latest_feedback}
            
            Please revise the content addressing this feedback:
            """)
        ])
        
        response = synthesizer_model.invoke(prompt.format_messages())
        
        # Update the refined content
        refined_content = response.content
        section_content = state.get("section_content", {})
        section_content[current_section["title"]] = refined_content
        
        return {
            **state,
            "section_content": section_content,
            "status": "reviewing"
    }

# 5. Finalizer Node - Compiles the final document
def compile_document(state: DocumentState) -> DocumentState:
    sections = state["sections"]
    section_content = state["section_content"]

    document = f"# {state['requirements']} Documentation\n\n"

    for section in sections:
        title = section["title"]
        content = section_content.get(title, "").strip()

        if content:
            document += f"\n## {title}\n\n{content}\n"
        else:
            document += f"\n## {title}\n\n⚠️ (No content generated)\n"

    return {**state, "complete_document": document, "status": "complete"}


In [5]:
# Create the graph
def create_workflow_graph():
    # Define the graph
    workflow = StateGraph(DocumentState)
    
    # Add nodes
    workflow.add_node("planner", plan_document)
    workflow.add_node("synthesizer", generate_section_content)
    workflow.add_node("reviewer", review_section)
    workflow.add_node("refiner", refine_section)
    workflow.add_node("finalizer", compile_document)
    
    # Add edges based on state transitions
    workflow.add_edge("planner", "synthesizer")
    workflow.add_conditional_edges(
        "synthesizer",
        lambda state: state["status"],
        {
            "reviewing": "reviewer",
            "complete": "finalizer"
        }
    )
    workflow.add_conditional_edges(
        "reviewer",
        lambda state: state["status"],
        {
            "refining": "refiner",
            "generating": "synthesizer",
            "complete": "finalizer"
        }
    )
    workflow.add_edge("refiner", "reviewer")
    workflow.add_edge("finalizer", END)
    
    # Set the entry point
    workflow.set_entry_point("planner")
    
    return workflow.compile()

# Main execution function
def generate_documentation(requirements: str) -> str:
    # Initialize state
    initial_state = {
        "requirements": requirements,
        "sections": [],
        "section_index": 0,
        "current_section": {},
        "section_content": {},
        "feedback": [],
        "iteration_count": 0,
        "complete_document": "",
        "status": "planning"
    }
    
    # Create and run the graph
    graph = create_workflow_graph()
    config = {"recursion_limit": 50}  # Adjust based on your process needs
    final_state = graph.invoke(initial_state, config=config)
    
    return final_state["complete_document"]


In [6]:
# Example usage
if __name__ == "__main__":
    requirements = """
    Create comprehensive documentation for a Python-based data processing API that handles CSV and JSON data.
    The API should support data validation, transformation, and export to multiple formats.
    Include installation instructions, API reference, and usage examples.
    """
    
    result = generate_documentation(requirements)
    
    # Save the result to a file
    with open("generated_documentation.md", "w") as f:
        f.write(result)
    
    print("Documentation generated successfully!")


🔎 Planner LLM Response:
 # Technical Documentation Outline for Python-Based Data Processing API

## 1. Introduction
**Purpose:** Provide an overview of the API, including its main functionalities and use cases.
- Brief description of the API, its capabilities, and the types of data it handles (CSV and JSON).
- Overview of key features such as data validation, transformation, and export capabilities.
- Use case examples where the API could be beneficial (e.g., data cleaning, data integration tasks).

## 2. Getting Started
**Purpose:** Guide users on how to install and configure the API.
- **System Requirements:** 
  - List of compatible operating systems and required software (e.g., Python version, dependencies).
- **Installation Instructions:**
  - Step-by-step procedure for installing the API using package managers (e.g., pip).
  - Instructions for setting up the API in a development environment.
- **Configuration:**
  - Guide on configuring the API settings, if applicable.
  - Detai

KeyboardInterrupt: 